# Notebook #4 - Pathfinder Timing and Profiling 

Initial steps: We must import specific environment variables to point to the user's notebook code director and to the Lucata tools.

In [ ]:
import os
os.environ["USER_NOTEBOOK_CODE"]=os.environ["HOME"]+"/pearc-tutorial/code"
os.environ["PATH"]=os.pathsep.join(["/tools/emu/pathfinder-sw/21.06/bin",os.environ["PATH"]])

This notebook goes along with the [Lucata profiling and timing slides](https://github.com/gt-crnch-rg/pearc-tutorial-2021/blob/main/slides/lucata_tutorial/03_Lucata_Pathfinder_Tutorial_Performance_Profiling.pdf), so please follow along with the slides for a supplemental resource. 

## Lucata Timing Hooks

The Lucata toolchain includes timing hooks that can be specified to provide timing information with the Lucata simulation profiler, `emusim_profile`. These can be added to regions of the code to just provide timing and profiling information for that region of code. Here we annotate the computation part of SAXPY but place the timing hooks after the initialization phase.


```c
//timing-hooks-saxpy.c
#include <stdio.h>
#include <stdlib.h>
#include <cilk/cilk.h>

#include "memoryweb.h"
#include <emu_c_utils/emu_c_utils.h>

void saxpy(long n, float a, float *x, float *y)
{
  for (long i = 0; i < n; i++)
    y[i] += a * x[i];
}

int main(int argc, char **argv)
{
  long num = atol(argv[1]); // number blocks
  long size = atol(argv[2]); // block size
  float aval = atof(argv[3]); // constant
  float **x = mw_malloc2d(num, size * sizeof(*x));
  float **y = mw_malloc2d(num, size * sizeof(*y));

  for (long j = 0; j < num; j++) {
    for (long i = 0; i < size; i++) {
      x[j][i] = j * size + i; y[j][i] = 0;
    }
  }
  
  hooks_region_begin("example");
  for (long i = 0; i < num; i++) {
    cilk_spawn_at (y[i]) saxpy(size, aval, x[i], y[i]);
  }
  cilk_sync;
    
  double time_ms = hooks_region_end();
  printf("time (ms) = %lf\n", time_ms);
}
```

### Profiling Output
We can then compile and profile the code. This will generate a separate folder "saxpy_profile" with HTML output files that you can then investigate in further detail. 

Note that this process might take a long time! For this reason it is important to only scope the region of interest in your code that you want to gather statistics for. Here we run with 1 node to shorten the runtime of the profiler.

In [6]:
%%bash
emu-cc -o timing-hooks-saxpy.mwx timing-hooks-saxpy.c -l emu_c_utils

In [7]:
%%bash
emusim_profile saxpy_profile --total_nodes 1 -- timing-hooks-saxpy.mwx 8 128 5.0 

Generating profile in saxpy_profile/timing-hooks-saxpy
emusim.x  --total_nodes 1
timing-hooks-saxpy.mwx 8 128 5.0
Start untimed simulation with local date and time= Mon Jul 19 03:24:13 2021

End untimed simulation with local date and time= Mon Jul 19 03:24:13 2021

SysC Enumeration done. Program launching...
Simulation @0 s with local date and time= Mon Jul 19 03:24:13 2021

Simulation @1 ms with local date and time= Mon Jul 19 03:24:20 2021

Simulation @2 ms with local date and time= Mon Jul 19 03:24:26 2021

Simulation @3 ms with local date and time= Mon Jul 19 03:24:32 2021

{"region_name":"example","time_ms":0.43,"ticks":75041}
Simulation @4 ms with local date and time= Mon Jul 19 03:24:39 2021

time (ms) = 0.428806

Info: /OSCI/SystemC: Simulation stopped by user.



        SystemC 2.3.3-Accellera --- Jun 22 2021 17:09:43
        Copyright (c) 1996-2018 by all Contributors,
        ALL RIGHTS RESERVED
Traceback (most recent call last):
  File "/net/tools/emu/pathfinder-sw/21.06/bin/make_uis_plots.py", line 7, in <module>
    import parser as emup
  File "/net/tools/emu/pathfinder-sw/21.06/bin/parser.py", line 1, in <module>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'


CalledProcessError: Command 'b'emusim_profile saxpy_profile --total_nodes 1 -- timing-hooks-saxpy.mwx 8 128 5.0 \n'' returned non-zero exit status 1.

### Postcript

Once we've finished our testing, we can clean up some of the logfiles that we used for this example with `make clean`. Uncomment the following line to clean this directory.

In [ ]:
#!make clean